In [2]:
questions=["age","history","parenchymal_distortion","nodules_echo_size"]


def fix_brackets_spaces(texto):
    ''' 
        Introduce espacios por delante y por detrás de los paréntesis.
        Esta medida mejora el tokenizado de Spacy
    '''
    
    texto = re.sub(r'([(\[¿!])', r' \1', texto)
    texto = re.sub(r'([)\]?¡])', r'\1 ', texto)
                
    return texto
    
def preprocess_text(text):
    text= fix_brackets_spaces(text)
    return text


def flatten_and_filter_dataset(ground_truth,reports):
    """
    Esta función toma un conjunto de datos en el formato original (con estructura jerárquica)
    y devuelve un conjunto de datos plano, donde cada entrada tiene un solo `context`, `question` y `answer`.
    
    Argumentos:
        dataset: Un conjunto de datos en formato original (puede ser train, validation, test).
    
    Retorno:
        Un conjunto de datos de Hugging Face en formato plano, con solo ejemplos completos.
    """
    # Lista para almacenar ejemplos en formato plano
    flattened_examples = {}
    examples_raw={}
    targets={}
    questions_examples={}
    val_data={}
    question_tipo={}
    previous_message_answer_tipo={}
    options_tipo={}
    answers_tipo={}
    j=0
    
    question_tipo["age"]= "does the patient's age appear in the following breast medical report?"
    previous_message_answer_tipo["age"]="search for numbers, but do not mistake it with the age of a familiar. If a number appears without any context between two dots it is surely the age."
    

    question_tipo["history"]= "does the patient have any non-familiar history in the following breast medical report?"
    previous_message_answer_tipo["history"]="check for the history at the beginning of the report. Normally it is a previous biopsy result, mastectomy or cancer. It normally starts with 'history of ...'."

    question_tipo["parenchymal_distortion"]= "does the following breast medical report mention any parenchymal distortion or asymmetry in the mammography exam?"
    previous_message_answer_tipo["parenchymal_distortion"]="If it has any it will appear in the results of the mammography exam using the words distortion, asymmetry or sometimes it can also be surgical changes."
    

    # question_tipo["nodules_echo_location_1"]= "In which location is the first nodule described in the ultrasound exam of the following breast medical report?"
    # previous_message_answer_tipo["nodules_echo_location_1"]="do not consider if a nodule is described in the mammography exam or if it is in the axilla. If the nodule is mentioned previously in the mammography, the location can be found also there. sometimes the breast location of the tumour may be written in a different part than the quadrant. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
    
    
    question_tipo["nodules_echo_size_1"]= "what is the size of the first nodule described in the ultrasound exam of the following breast medical report?"
    previous_message_answer_tipo["nodules_echo_size_1"]="do not consider if a nodule is described in the mammography exam. The localization, echogenicity and size of the nodules are normally said. More than one nodule can be described simultaneously ('several', 'two', 'three', etc.). If the number is unspecified, only the ones with size will be considered. Sometimes it can also say 'similar characteristics' or 'similar to the previous', consider in this case the answer to the previous tumour."
    
    # Iterar sobre cada ejemplo en el conjunto de datos original
    for i, report in enumerate(reports["informes_ingles"]):
        informe=preprocess_text(report)
        key=reports["keys"][i]
        
        if key not in ground_truth.index:
            continue
        answer_tipo={}
        row=ground_truth.loc[key]
        age=str(row["Age"])
        # Verificar si el ejemplo tiene preguntas
        if age.isdigit():
            answer_tipo["age"]=age
            
        else:
            answer_tipo["age"]="no response"

        if row["Biopsy_report"]!="Yes" and row["Ganglio_report"]!="Yes":
            history=row["Other_history"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(history,str) or history=="No":
                answer_tipo["history"]="no response"           
            else:
                answer_tipo["history"]=history


            #Parenchymal distortion
            parenchymal_distortion=row["parenchymal_distortions_asymmetry"]
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(parenchymal_distortion,str) or parenchymal_distortion.lower()=="no":
                answer_tipo["parenchymal_distortion"]="no response"        
            else:
                answer_tipo["parenchymal_distortion"]=parenchymal_distortion.lower()

            #Nodules echo
            nodules_echo_num=row["Nodules_eco"]
            
            # Verificar si el ejemplo tiene preguntas
            if not isinstance(nodules_echo_num,str) and not isinstance(nodules_echo_num,int):
                nodules=False
            elif isinstance(nodules_echo_num,str) and nodules_echo_num=="No":
                nodules=False
            else:
                nodules=True
            if nodules:
                # nodules_echo_location=row["Location_eco_1"]
                # # Verificar si el ejemplo tiene preguntas
                # if not isinstance(nodules_echo_location,str):
                #     answer_tipo["nodules_echo_location_1"]="no response"
                # else:
                #     answer_tipo["nodules_echo_location_1"]=nodules_echo_location.lower()
                #Nodules echo size
                nodules_echo_size=row["size_eco_1"]
                # Verificar si el ejemplo tiene preguntas
                if not isinstance(nodules_echo_size,str):
                    answer_tipo["nodules_echo_size_1"]="no response"         
                else:
                    answer_tipo["nodules_echo_size_1"]=nodules_echo_size
                

            
        for tipo in answer_tipo:
            #Si está el tipo en las respuestas que hemos recogido lo metemos a la base de datos.
            key_tipo=key+"_"+tipo
            if key_tipo in flattened_examples:
                continue
                key_tipo=key_tipo+"_copy"
            examples_raw[key_tipo]=report
            answer=str(answer_tipo[tipo])
            
            questions_examples[key_tipo]=question_tipo[tipo]
            inputs_tipo = " Additional information: "+ previous_message_answer_tipo[tipo]+ " Context: " + informe
            flattened_examples[key_tipo]=inputs_tipo
            targets[key_tipo]=answer.strip()
        
        
        
        # if answer==word_to_idx_out["other"]:
        #     for j in range(2):
        #         examples_raw[key+"_copy"+str(j)]=report
        #         flattened_examples[key+"_copy"+str(j)]=informe
        #         targets[key+"_copy"+str(j)]=answer
            
        
            
        
    # flattened_examples=pd.DataFrame.from_dict(flattened_examples,orient='index')
    # targets=pd.DataFrame.from_dict(targets,orient='index')
    return flattened_examples,questions_examples,targets,examples_raw


inputs,questions_examples,targets,examples_raw = flatten_and_filter_dataset(ground_truth,report_data)  
dataset_final=pd.DataFrame.from_dict(inputs,orient='index')
questions_examples=pd.DataFrame.from_dict(questions_examples,orient='index')

print(len(dataset_final),len(questions_examples),len(targets))

targets=pd.DataFrame.from_dict(targets,orient='index')
dataset_final.columns=["text"]

dataset_final["question"]=questions_examples

targets.columns=["label"]


tokenizer = AutoTokenizer.from_pretrained(model_name)
def visualize_errors(valid_dataset,valid_targets,validation_predictions,keys):
    # Crear un DataFrame con los textos originales, las etiquetas reales y las predicciones
    results_df = pd.DataFrame({
        'key':list(keys),
        'Text': list(valid_dataset),  # Usamos los textos originales
        'True Label': list(valid_targets),
        'Predicted Label': list(validation_predictions)
    })
    
    # Filtrar los ejemplos en los que el modelo falló
    errors_df = results_df[results_df['True Label'] != results_df['Predicted Label']]
    
    for ind,row in errors_df.iterrows():
        print(row["key"])
        print("EJEMPLO")
        print(row["Text"])
        print("PREDICTED")
        print(row["Predicted Label"])
        print("TRUE")
        print(row["True Label"])

def tokenize_function(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["text"],
        truncation=True,
        max_length=512,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = tokenized_examples.pop("offset_mapping")
    start_positions = []
    end_positions = []

    context_texts = examples["text"]
    answer_texts = examples["label"]
    errors=[]

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        context = context_texts[i]
        answer = answer_texts[i]

        # Default to CLS for no response
        if answer == "no response" or answer.strip() == "":
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        # Lowercase match to avoid case mismatch
        start_char = context.lower().find(answer.lower())
        if start_char == -1:
            print(i)
            errors.append(i)
            print(f"[WARNING] Could not find answer: '{answer}' in context:\n{context}")
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        end_char = start_char + len(answer)

        # Now find token positions
        start_pos = None
        end_pos = None
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end and start_pos is None:
                start_pos = idx
            if start < end_char <= end:
                end_pos = idx
                break

        if start_pos is None or end_pos is None:
            # Fallback if something failed
            print(f"[WARNING] Failed to align answer '{answer}' in context")
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_positions.append(start_pos)
            end_positions.append(end_pos)
        
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples
    
def test_train_split(i, X, Y):
    random.seed(1)
    # Agrupar ejemplos originales y sus copias
    def obtener_grupo(nombre):
        # Extraer el nombre base eliminando '_copia' y cualquier número posterior
        if '_copy' in nombre:
            return nombre.split('_copy')[0]

        return nombre

    # Crear un DataFrame temporal para manejar los índices
    agrupaciones = pd.DataFrame(index=X.index)
    agrupaciones['grupo'] = agrupaciones.index.map(obtener_grupo)
    # Obtener una lista de grupos únicos
    grupos_unicos = agrupaciones['grupo'].unique()

    # Dividir grupos en 10 folds
    cv_tamaño = math.ceil(len(grupos_unicos) / 10)
    test_grupos = grupos_unicos[i * cv_tamaño:min(len(grupos_unicos), (i + 1) * cv_tamaño)]
    

    # Filtrar conjuntos de prueba y entrenamiento basados en los grupos
    test_indices = test_grupos
    train_indices = agrupaciones[~agrupaciones['grupo'].isin(test_grupos)].index
    
    # Seleccionar los datos de entrenamiento y prueba
    test = X.loc[test_indices].sort_index()
    test_y = Y.loc[test_indices].sort_index()
    ind=list(test.index)
    
    train = X.loc[train_indices].sort_index()

    # Barajar los datos de entrenamiento para evitar sesgos
    train = train.sample(frac=1, random_state=1)

    train_y = Y.loc[train.index]

    train["label"]=train_y["label"]
    test["label"]=test_y["label"]
    train_ind=list(train.index)
    # Devolver los conjuntos
    return train, test, ind,train_ind


def predict_indexes(pred):

    # Extract logits from predictions
    start_logits, end_logits = pred.predictions

    
    # Get the best start and end indices
    start_indexes = np.argmax(start_logits, axis=1)
    end_indexes = np.argmax(end_logits, axis=1)

    
    return start_indexes, end_indexes

def extract_answer_from_tokens(tokenized_inputs, start_index, end_index):
    """
    Extracts the predicted answer using tokenized input and index positions.
    
    Args:
        tokenized_inputs: The tokenized dataset
        start_index: Predicted start position
        end_index: Predicted end position
    
    Returns:
        Extracted text or "No response" if CLS token is chosen
    """
    # Convert token IDs back to words
    tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
    
    
    # If CLS token is chosen (indicating no answer)
    if start_index == 0 or end_index == 0 or start_index > end_index:
        return "no response"

    # Extract the predicted text
    answer_tokens = tokens[start_index:end_index+1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    

    
    return answer


def cross_validation(X, Y):
    import torch
    from transformers import AutoModelForQuestionAnswering, Trainer
    from datasets import Dataset
    from sklearn.metrics import accuracy_score, classification_report

    random.seed(1)
    np.random.seed(1)
    
    predicted = []
    tested = []
    acc_cv = []
    ind_cv={tipo:[] for tipo in questions}
    preds_category_cv={tipo:[] for tipo in questions}
    labels_category_cv={tipo:[] for tipo in questions}
    accuracies_cv={tipo:[] for tipo in questions}

    for i in range(10):
        train, test, ind,train_ind = test_train_split(i, X, Y)
        print(len(train), len(test))

        model = AutoModelForQuestionAnswering.from_pretrained(
            model_name
        )
        
        

        train_data = Dataset.from_pandas(train)
        test_data = Dataset.from_pandas(test)

        # Tokenize datasets

        train_data= train_data.map(tokenize_function, batched=True)
        train_data = train_data.rename_column("label", "labels")
        train_data = train_data.remove_columns(["text"])
        train_data.set_format("torch")


        print("ERRORES TEST")
        test_data = test_data.map(tokenize_function, batched=True)
        test_data = test_data.rename_column("label", "labels")
        test_data = test_data.remove_columns(["text"])
        test_data.set_format("torch")
        # for j, indice in enumerate(train_ind):
        #     print(j,indice)

        
            

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_data,
            eval_dataset=test_data
        )

        trainer.train()

        # **Make Predictions**
        pred = trainer.predict(test_data)
        start_indexes, end_indexes = predict_indexes(pred)

        answers = []
        for j in range(len(start_indexes)):
            # Extract answer from tokenized test data
            tokenized_example = test_data[j]
            answer = extract_answer_from_tokens(tokenized_example, start_indexes[j], end_indexes[j])
            answers.append(answer)
            # print(f"Extracted Answer {j+1}: {answer}")

        test_data={}
        test_label={}
        for tipo in questions:
            #Primero creamos la lista y luego vemos que no esté vacía para hacer el stack
            data=[output for j,output in enumerate(answers) if re.search(rf"_{tipo}(_\d+)?$", ind[j])]
            print(data)
            
            if data:
                test_data[tipo]=data
                
                del data
                

            labels=[output for j,output in enumerate(test["label"])  if re.search(rf"_{tipo}(_\d+)?$", ind[j])]
            
            if labels:
                #Convertimos a token y luego str para que tenga el mismo formato que las respuestas.
                test_label[tipo] = [
                tokenizer.convert_tokens_to_string(tokenizer.tokenize(lab)).strip() 
                for lab in labels
            ]
                print(test_label[tipo])
                
        ind_fold={tipo: [key for key in ind if re.search(rf"_{tipo}(_\d+)?$", key)] for tipo in questions}
        
        for tipo in questions:
            ind_cv[tipo].append(ind_fold[tipo])
        # **Ensure Label Comparison Works Correctly**
        ground_truths = test["label"].tolist()

        for tipo in questions:
            if tipo in test_data:
                acc = accuracy_score(test_label[tipo], test_data[tipo])
                accuracies_cv[tipo].append(acc)
                print(f"Accuracy for {tipo}: {acc:.4f}")
                preds_category_cv[tipo]+=test_data[tipo]
                labels_category_cv[tipo]+=test_label[tipo]

        # **Evaluate Accuracy**
        accuracy = accuracy_score(ground_truths, answers)
        

       

        # **Visualize Errors**
        valid_dataset = [examples_raw[key] for key in ind]
        visualize_errors(valid_dataset, np.array(ground_truths), answers, ind)

    accuracies=[]    
    output_dic={}
    output_dic_t={}
    for tipo in questions:

        ind_cv[tipo]=np.concatenate(ind_cv[tipo])
        acc = accuracy_score(labels_category_cv[tipo], preds_category_cv[tipo])
        print(f"Accuracy for {tipo}: {acc:.4f}")
        print(f"Accuracy std for {tipo}: {np.std(accuracies_cv[tipo])}")
        accuracies.append(acc)
        output_dic[tipo]={ind:preds_category_cv[tipo][i] for i, ind in enumerate(ind_cv[tipo])}
        output_dic_t[tipo]={ind:labels_category_cv[tipo][i] for i, ind in enumerate(ind_cv[tipo])}
        with open(f"../Generativos/truth_dic/{tipo}.pkl", "wb") as file:
            pickle.dump(output_dic_t[tipo], file)
        
    
    return accuracies,output_dic,output_dic_t

NameError: name 'ground_truth' is not defined